In [1]:
from bdb import effective
from glob import glob
from telegram import InlineKeyboardButton, InlineKeyboardMarkup, Update
from sqlite3 import Error
from random import *
from random import choice
import matplotlib
import random
import telegram
import logging
import sqlite3
import os
import csv
from prebuiltSets import *
from prebuiltSets import void
from prebuiltSets import forest_location_list
from prebuiltSets import dungeon_location_list
from prebuilt import *
from prebuilt import easy_attack_list
from telegram.ext import (
    Updater,
    CallbackQueryHandler,
    CommandHandler,
    ConversationHandler,
    CallbackContext, 
    MessageHandler, 
    Filters
    )

logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                     level=logging.INFO)

#API_TOKEN - is your telegram bot token. Just copy it in.
API_TOKEN = os.environ.get('TOKEN')
print(API_TOKEN)
#CHAT_ID - is your telegram ID. It is nessecary for notifications.
CHAT_ID = os.environ.get('CHAT_ID')
print(CHAT_ID)

updater = Updater(API_TOKEN)
dispatcher = updater.dispatcher
job_queue = updater.job_queue

action_quotes = [
"Fight! I cannot understand what is going on, but I feel what I need to do.",
"I NEED MOVE!",
"Time to think...",
"My turn...",
"I need to spectate him, or I'll be dead...",
"Khu... khu... I need to choose.",
"It'll crash me!",
"Don't panic...",
"Action time...",
"I'll crash it!",
"I need to hit, until it's too late!",
"Defense... I need to be aware.",
"I'm sorry for that."
]

conn = sqlite3.connect('test.db', check_same_thread=False)
print("Opened database successfully")
cursor = conn.execute("SELECT ID,NAME,SEX,RACE,CLASS from USERS")
#Inventory
user_data = dict()
user_inventory = list()
user_inventory_names = list()
user_position = 'Right side'
user_health = {
    'headHealth':5,
    'headMaxHealth':5,
    'headDefense':0,
    'neckHealth':3,
    'neckMaxHealth':3,
    'neckRightDefense':0,
    'neckLeftDefense':0,
    'torsoHealth':10,
    'torsoMaxHealth':10,
    'torsoRightDefense':0,
    'torsoLeftDefense':0,
    'pelvisHealth':10,
    'pelvisMaxHealth':10,
    'pelvisDefense':0,
    'rshoulderHealth':5,
    'rshoulderMaxHealth':5,
    'rshoulderRightDefense':0,
    'rshoulderLeftDefense':0,
    'lshoulderHealth':5,
    'lshoulderMaxHealth':5,
    'lshoulderRightDefense':0,
    'lshoulderLeftDefense':0,
    'relbowHealth':5,
    'relbowMaxHealth':5,
    'relbowRightDefense':0,
    'relbowLeftDefense':0,
    'lelbowHealth':5,
    'lelbowMaxHealth':5,
    'lelbowRightDefense':0,
    'lelbowLeftDefense':0,
    'rhandHealth':5,
    'rhandMaxHealth':5,
    'rhandRightDefense':0,
    'rhandLeftDefense':0,
    'lhandHealth':5,
    'lhandMaxHealth':5,
    'lhandRightDefense':0,
    'lhandLeftDefense':0,
    'rlegHealth':7,
    'rlegMaxHealth':7,
    'rlegRightDefense':0,
    'rlegLeftDefense':0,
    'llegHealth':7,
    'llegMaxHealth':7,
    'llegRightDefense':0,
    'llegLeftDefense':0,
    'rkneeHealth':5,
    'rkneeMaxHealth':5,
    'rkneeRightDefense':0,
    'rkneeLeftDefense':0,
    'lkneeHealth':5,
    'lkneeMaxHealth':5,
    'lkneeRightDefense':0,
    'lkneeLeftDefense':0,
    'rfootHealth':7,
    'rfootMaxHealth':7,
    'rfootRightDefense':0,
    'rfootLeftDefense':0,
    'lfootHealth':7,
    'lfootMaxHealth':7,
    'lfootRightDefense':0,
    'lfootLeftDefense':0
}
user_grabbed = {
    'rhandGrabbed': 0,
    'lhandGrabbed': 0,
    'rlegGrabbed': 0,
    'llegGrabbed': 0
}

user_equipment = {
    'rwield':None,
    'lwield':None,
    'helmet':None,
    'rhelmet':None,
    'neck':None,
    'rneck':None,
    'chest':None,
    'rchest':None,
    'bag':None,
    'belt':None,
    'rbelt':None,
    'pants':None,
    'rpants':None,
    'rleg':None,
    'lleg':None,
    'rknee':None,
    'lknee':None,
    'rfoot':None,
    'lfoot':None,
    'rshoulder':None, 
    'lshoulder':None,
    'relbow':None,
    'lelbow':None,
    'rhand':None,
    'lhand':None,
    'rring1':None,
    'lring1':None,
    'rring2':None,
    'lring2':None,
    'rring3':None,
    'lring3':None,
    'rring4':None,
    'lring4':None,
    }
user_tdamage = 0
user_maxBalance = 3
user_Balance = 0
user_target = ''

current_location = dict()

clear = lambda: os.system('cls')
turn_token = 0

enemy_Balance = 0
enemy_maxBalance = 3
enemy_isGrabbed = False
enemy_tdamage = 0
enemy_target = ''
enemy_target_list = ['head','neck','torso','pelvis','rshoulder','lshoulder','relbow','lelbow','rhand','lhand','rleg','lleg','rknee','lknee','rfoot','lfoot']
enemy_crittarget_list = ['head','neck','torso']
enemy_health =  {
    'headHealth':10,
    'headMaxHealth':10,
    'headDefense':0,
    'neckHealth':8,
    'neckMaxHealth':8,
    'neckRightDefense':0,
    'neckLeftDefense':0,
    'torsoHealth':15,
    'torsoMaxHealth':15,
    'torsoRightDefense':0,
    'torsoLeftDefense':0,
    'pelvisHealth':15,
    'pelvisMaxHealth':15,
    'pelvisDefense':0,
    'rshoulderHealth':10,
    'rshoulderMaxHealth':10,
    'rshoulderRightDefense':0,
    'rshoulderLeftDefense':0,
    'lshoulderHealth':10,
    'lshoulderMaxHealth':10,
    'lshoulderRightDefense':0,
    'lshoulderLeftDefense':0,
    'relbowHealth':10,
    'relbowMaxHealth':10,
    'relbowRightDefense':0,
    'relbowLeftDefense':0,
    'lelbowHealth':10,
    'lelbowMaxHealth':10,
    'lelbowRightDefense':0,
    'lelbowLeftDefense':0,
    'rhandHealth':10,
    'rhandMaxHealth':10,
    'rhandRightDefense':0,
    'rhandLeftDefense':0,
    'lhandHealth':10,
    'lhandMaxHealth':10,
    'lhandRightDefense':0,
    'lhandLeftDefense':0,
    'rlegHealth':12,
    'rlegMaxHealth':12,
    'rlegRightDefense':0,
    'rlegLeftDefense':0,
    'llegHealth':12,
    'llegMaxHealth':12,
    'llegRightDefense':0,
    'llegLeftDefense':0,
    'rkneeHealth':10,
    'rkneeMaxHealth':10,
    'rkneeRightDefense':0,
    'rkneeLeftDefense':0,
    'lkneeHealth':10,
    'lkneeMaxHealth':10,
    'lkneeRightDefense':0,
    'lkneeLeftDefense':0,
    'rfootHealth':12,
    'rfootMaxHealth':12,
    'rfootRightDefense':0,
    'rfootLeftDefense':0,
    'lfootHealth':12,
    'lfootMaxHealth':12,
    'lfootRightDefense':0,
    'lfootLeftDefense':0
}
enemy_grabbed = {
    'rhandGrabbed': 0,
    'lhandGrabbed': 0,
    'rlegGrabbed': 0,
    'llegGrabbed': 0
}
dead_counter = 0

#INVENTORY TWEAKS
def addItemToInventory():
    clear()
    if user_data['class'] == 'Adventurer':
        user_inventory.append(shirt)
        user_inventory.append(pants)
        user_inventory.append(smallAxe)

        user_equipment['rwield'] = smallAxe
        user_equipment['chest'] = shirt
        user_equipment['pants'] = pants
        #addItemToFile(user_inventory, clear =False)
        
        for i in user_inventory:
            user_inventory_names.append(i['name'])
        
        #print(user_inventory_names)
        #print(user_equipment['chest'])
        #print(user_data.items())
        #print(user_equipment['rwield']['name'])
        viewInventory()        

def viewInventory():  
    if user_equipment['chest'] != None:
        if user_equipment['chest']['type'] == 'shirt':
            user_health['torsoHealth'] += user_equipment['chest']['health']
            user_health['rshoulderHealth'] += user_equipment['chest']['health']
            user_health['lshoulderHealth'] += user_equipment['chest']['health']
            user_health['relbowHealth'] += user_equipment['chest']['health']
            user_health['lelbowHealth'] += user_equipment['chest']['health']
            user_health['rhandHealth'] += user_equipment['chest']['health']
            user_health['lhandHealth'] += user_equipment['chest']['health']
    
    if user_equipment['pants'] != None:
        if user_equipment['pants']['type'] == 'pants':
            user_health['pelvisHealth'] += user_equipment['pants']['health']
            user_health['rlegHealth'] += user_equipment['pants']['health']
            user_health['llegHealth'] += user_equipment['pants']['health']
            user_health['rkneeHealth'] += user_equipment['pants']['health']
            user_health['lkneeHealth'] += user_equipment['pants']['health']
            user_health['rfootHealth'] += user_equipment['pants']['health']
            user_health['lfootHealth'] += user_equipment['pants']['health']

    print(user_health)
    
def editInventoryItem():
    clear()

def deleteInventoryItem():
    pass

def addItemToFile(user_inventory: list, clear: bool):
    pass

def getInventoryItems():
    pass

def start(update: Update, context: CallbackContext) -> None:
    menu(update, context)

#MENU
#Main menu function (Summoned by player)
def menu(update: Update, context: CallbackContext) -> None:
    chat = update.effective_chat
    context.bot.send_photo(chat_id=chat.id, photo=open('E:\git\GolemBot\KoDDancingPuppets.png', 'rb'))
    keyboard = [
        [
            InlineKeyboardButton("Start Game", callback_data="startGame")
        ],
        [
            InlineKeyboardButton("Options", callback_data="options")
        ],
        [
            InlineKeyboardButton("Credits", callback_data="credits")
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.message.reply_text("""Kingdom of Death:
        Dancing Puppets
    
    Main Menu:""", reply_markup=reply_markup)

#Main menu function (automatic)
def amenu(update: Update, context: CallbackContext) -> None:
    keyboard = [
        [
            InlineKeyboardButton("Start Game", callback_data="startGame")
        ],
        [
            InlineKeyboardButton("Options", callback_data="options")
        ],
        [
            InlineKeyboardButton("Credits", callback_data="credits")
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.callback_query.message.reply_text("""Kingdom of Death:
        Dancing Puppets
    
    Main Menu:""", reply_markup=reply_markup)

#Start Game
def start_game(update: Update, context: CallbackContext) -> None:
    keyboard = [
        [
            InlineKeyboardButton("New Game", callback_data="newGame")
        ],
        [
            InlineKeyboardButton("Load Game", callback_data="loadGame")
        ],
        [
            InlineKeyboardButton("Back", callback_data="backToMainMenu")
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.callback_query.message.reply_text("Choose:", reply_markup=reply_markup)

#New Game
    #Name
def user_name(update: Update, context:CallbackContext):
    user_data.clear()
    user_inventory.clear()
    user_inventory_names.clear()
    conn.rollback()
    chat = update.effective_chat
    context.bot.send_message(chat_id=chat.id, text="""Hero, what is your name?
    
    (Use '/name' and your nickname to proceed)""")

    #User name
    dispatcher.add_handler(CommandHandler("name", user_name_input))
    #Name input
def user_name_input(update, context):

    user_data['name'] = ''.join(context.args)
    if user_data["name"] != '':
        conn.execute(f"INSERT INTO USERS (ID,NAME,SEX,RACE,CLASS) \
        VALUES ({update.message.from_user.id},'o', 'o', 'o', 'o' )");
        print('Users blanks created:', conn.total_changes)
        conn.execute(f"UPDATE USERS set NAME = ? where ID = ?", (user_data['name'], update.message.from_user.id));
        print('Users blanks changed:', conn.total_changes)
        user_sex(update, context)
        
    else: back_main_amenu(update, context)    
    #Sex
def user_sex(update: Update, context:CallbackContext):
    chat = update.effective_chat

    keyboard = [
        [
            InlineKeyboardButton("Hero (Male)", callback_data="male")
        ],
        [
            InlineKeyboardButton("Heroine(Female)", callback_data="female")
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.message.reply_text(f"Are you Hero or Heroine, {user_data['name']}?", reply_markup=reply_markup)
    #Race
def user_race(update: Update, context:CallbackContext):
    keyboard = [
        [
            InlineKeyboardButton("Human", callback_data="humanRace")
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.callback_query.message.reply_text("What are you?", reply_markup=reply_markup)
    
    #Starter Items
def starter_pack(update: Update, context:CallbackContext):
    keyboard = [
        # [
        #     InlineKeyboardButton("""I were a knight...  (Heavy armory and equipment)""", callback_data="knightSet")
        # ],
        # [
        #     InlineKeyboardButton("""I were a freelancer...  (Light leather armor and early real arms)""", callback_data="freelancerSet")
        # ],
        [
            InlineKeyboardButton("""I were an adventurer...    (You have Nothing, but small axe)""", callback_data="adventurerSet")
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.callback_query.message.reply_text("Whom were you?", reply_markup=reply_markup)

    #Finish character creation
def user_fin(update: Update, context:CallbackContext):
    keyboard = [
        [
            InlineKeyboardButton("""Yes, I am.  (Continue)""", callback_data="finishReg")
        ],
        [
            InlineKeyboardButton("""It's not me!     (Reset character creation)""", callback_data="newGame")
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.callback_query.message.reply_text(f"""I've scribbled everything down on this ceramic plate. Read and check all the info, {user_data['name']}. Is that you?
    
    Name: {user_data['name']}
    Sex: {user_data['sex']}
    Race: {user_data['race']}
    Class: {user_data['class']}
    
    Inventory:
    {user_inventory_names}""", reply_markup=reply_markup)

#GAMEPLAY
    #Location
def adventure(update: Update, context:CallbackContext):
    print(update.message.from_user.id)
    userID = update.message.from_user.id

    if user_data['name'] == ''.join(context.args):
        conn.execute(f"UPDATE USERS set SEX = ? where ID = ?", (user_data['sex'], userID));
        print('Users blanks changed:', conn.total_changes)
        conn.execute(f"UPDATE USERS set RACE = ? where ID = ?", (user_data['race'], userID));
        print('Users blanks changed:', conn.total_changes)
        conn.execute(f"UPDATE USERS set CLASS = ? where ID = ?", (user_data['class'], userID));
        print('Users blanks changed:', conn.total_changes)
        cursor = conn.execute("SELECT ID,NAME,SEX,RACE,CLASS from USERS")
        for row in cursor:    
            print("ID = ", row[0])
            print("NAME = ", row[1])
            print("SEX = ", row[2])
            print("RACE = ", row[3])
            print("CLASS = ", row[4])

        start_location(update, CallbackContext)
    else:
        update.message.reply_text(f'Try one more time...')

def start_location(update: Update, context:CallbackContext):
    keyboard = [
        [
            InlineKeyboardButton("""To the Void...    (Open location)""", callback_data="voidLoc")
        ],
        [
            InlineKeyboardButton("""To the woods...  (There will be an obstacle)""", callback_data="forestLoc")
        ],
        [
            InlineKeyboardButton("""To the dungeon...    (Claustrophobes, prepare!)""", callback_data="dungeonLoc")
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.message.reply_text("Where are you heading?", reply_markup=reply_markup)

def restart_location(update: Update, context:CallbackContext):
    keyboard = [
        [
            InlineKeyboardButton("""To the Void...    (Open location)""", callback_data="voidLoc")
        ],
        [
            InlineKeyboardButton("""To the woods...  (There will be an obstacle)""", callback_data="forestLoc")
        ],
        [
            InlineKeyboardButton("""To the dungeon...    (Claustrophobes, prepare!)""", callback_data="dungeonLoc")
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.callback_query.message.reply_text("Where are you heading?", reply_markup=reply_markup)

    # Location generator
def location_gen(update, context) -> None:
    global current_location
    global user_position
    global turn_token
    global user_target
    global enemy_target
    global user_health
    global enemy_health
    global user_tdamage
    global enemy_tdamage
    global enemy_Balance
    global enemy_maxBalance
    global enemy_grabbed
    #print(enemy_health)
    user_health['headHealth'] = user_health['headMaxHealth']
    user_health['neckHealth'] = user_health['neckMaxHealth']
    user_health['torsoHealth'] = user_health['torsoMaxHealth']
    user_health['pelvisHealth'] = user_health['pelvisMaxHealth']
    user_health['rshoulderHealth'] = user_health['rshoulderMaxHealth']
    user_health['lshoulderHealth'] = user_health['lshoulderMaxHealth']
    user_health['relbowHealth'] = user_health['relbowMaxHealth']
    user_health['lelbowHealth'] = user_health['lelbowMaxHealth']
    user_health['rhandHealth'] = user_health['rhandMaxHealth']
    user_health['lhandHealth'] = user_health['lhandMaxHealth']
    user_health['rlegHealth'] = user_health['rlegMaxHealth']
    user_health['llegHealth'] = user_health['llegMaxHealth']
    user_health['rkneeHealth'] = user_health['rkneeMaxHealth']
    user_health['lkneeHealth'] = user_health['lkneeMaxHealth']
    user_health['rfootHealth'] = user_health['rfootMaxHealth']
    user_health['lfootHealth'] = user_health['lfootMaxHealth']
    user_maxBalance = 3
    user_Balance = 0

    enemy_health['headHealth'] = enemy_health['headMaxHealth']
    enemy_health['neckHealth'] = enemy_health['neckMaxHealth']
    enemy_health['torsoHealth'] = enemy_health['torsoMaxHealth']
    enemy_health['pelvisHealth'] = enemy_health['pelvisMaxHealth']
    enemy_health['rshoulderHealth'] = enemy_health['rshoulderMaxHealth']
    enemy_health['lshoulderHealth'] = enemy_health['lshoulderMaxHealth']
    enemy_health['relbowHealth'] = enemy_health['relbowMaxHealth']
    enemy_health['lelbowHealth'] = enemy_health['lelbowMaxHealth']
    enemy_health['rhandHealth'] = enemy_health['rhandMaxHealth']
    enemy_health['lhandHealth'] = enemy_health['lhandMaxHealth']
    enemy_health['rlegHealth'] = enemy_health['rlegMaxHealth']
    enemy_health['llegHealth'] = enemy_health['llegMaxHealth']
    enemy_health['rkneeHealth'] = enemy_health['rkneeMaxHealth']
    enemy_health['lkneeHealth'] = enemy_health['lkneeMaxHealth']
    enemy_health['rfootHealth'] = enemy_health['rfootMaxHealth']
    enemy_health['lfootHealth'] = enemy_health['lfootMaxHealth']
    enemy_maxBalance = 3
    enemy_Balance = 0
    dead_counter = 0

    enemy_grabbed['lhandGrabbed'] = 0
    enemy_grabbed['rhandGrabbed'] = 0 
    enemy_grabbed['llegGrabbed'] = 0 
    enemy_grabbed['rlegGrabbed'] = 0

    turn_token = 0

    keyboard = [
        [
            InlineKeyboardButton("FIGHT", callback_data="enterCombat")
        ]
    ]

    print(current_location)

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.callback_query.message.reply_text("Brawl seems inevitable...", reply_markup=reply_markup)

    #COMBAT
def action(update: Update, context: CallbackContext) -> None:
    keyboard = [
        [
            InlineKeyboardButton("Combat", callback_data="combatAction")
        ],
        [
            InlineKeyboardButton("Skills", callback_data="skillAction")
        ],
        [
            InlineKeyboardButton("Items", callback_data="itemsAction")
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.callback_query.message.reply_text(f"""({user_data['name']}'s turn)
    
    {random.choice(action_quotes)}""", reply_markup=reply_markup)
def combat_hand(update: Update, context: CallbackContext) -> None:
    rightHand = user_equipment['rwield']
    leftHand = user_equipment['lwield']
    keyboard = []
    
    if rightHand != None:
        keyboard.append([InlineKeyboardButton(f"Right hand: ({rightHand['name']})", callback_data="rightHand")])
    if leftHand != None:
        keyboard.append([InlineKeyboardButton(f"Left hand: ({leftHand['name']})", callback_data="leftHand")])
    if rightHand == None:
        keyboard.append([InlineKeyboardButton(f"Right hand", callback_data="rightHand")])
    if leftHand == None:
        keyboard.append([InlineKeyboardButton(f"Left hand", callback_data="leftHand")])

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.callback_query.message.reply_text("I can hit with this... ", reply_markup=reply_markup)
def combat_rightHit(update: Update, context: CallbackContext) -> None:
    rightHand = user_equipment['rwield']
    keyboard = []

    if rightHand == None:
        keyboard.append([InlineKeyboardButton('Direct punch (1 DMG)', callback_data = 'punch')])
    else:    
        if rightHand['type'] == 'axe':
            keyboard.append([InlineKeyboardButton(f"Direct hit ({rightHand['damage']} DMG)", callback_data="rightDirectHit")])
            keyboard.append([InlineKeyboardButton(f"Blunt hit ({rightHand['damage'] - 2} DMG)", callback_data="rightBluntHit")])
            keyboard.append([InlineKeyboardButton(f"Upper hit ({rightHand['damage'] + 2} DMG)", callback_data="rightUpperHit")])
    
    reply_markup = InlineKeyboardMarkup(keyboard)
    update.callback_query.message.reply_text("Choose attack type:", reply_markup=reply_markup)

def combat_leftHit(update: Update, context: CallbackContext) -> None:
    leftHand = user_equipment['lwield']
    keyboard = []

    if leftHand == None:
        keyboard.append([InlineKeyboardButton('Direct punch (1 DMG)', callback_data = 'punch')])
    else:
        if leftHand['type'] == 'axe':
            keyboard.append([InlineKeyboardButton(f'Direct hit ({leftHand["damage"]} DMG)', callback_data="leftDirectHit")])
            keyboard.append([InlineKeyboardButton(f'Blunt hit ({leftHand["damage"] - 2} DMG)', callback_data="leftBluntHit")])
            keyboard.append([InlineKeyboardButton(f"Upper hit ({leftHand['damage'] + 2} DMG)", callback_data="leftUpperHit")])
        #elif leftHand['type'] == 'spear':
        #    keyboard.append([InlineKeyboardButton(f'Pierce ({leftHand["damage"]} DMG)', callback_data="pierceHit")])
        #    keyboard.append([InlineKeyboardButton(f'Blunt hit ({leftHand["damage"] - 4} DMG)', callback_data="spearBluntHit")])

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.callback_query.message.reply_text("Choose attack type:", reply_markup=reply_markup)

def combat_direction(update: Update, context: CallbackContext) -> None:
    keyboard = [
        [InlineKeyboardButton(f'Head', callback_data="headTarget")],
        [InlineKeyboardButton(f'Neck', callback_data="neckTarget")],
        [
            InlineKeyboardButton(f'Right shoulder', callback_data="rshoulderTarget"),
            InlineKeyboardButton(f'Torso', callback_data="torsoTarget"),
            InlineKeyboardButton(f'Left shoulder', callback_data="lshoulderTarget")
        ],
        [
            InlineKeyboardButton(f'Right elbow', callback_data="relbowTarget"),
            InlineKeyboardButton(f'Torso', callback_data="torsoTarget"),
            InlineKeyboardButton(f'Left elbow', callback_data="lelbowTarget")
        ],
        [
            InlineKeyboardButton(f'Right hand', callback_data="rhandTarget"),
            InlineKeyboardButton(f'Torso', callback_data="torsoTarget"),
            InlineKeyboardButton(f'Left hand', callback_data="lhandTarget")
        ],
        [
            InlineKeyboardButton(f'Right leg', callback_data="rlegTarget"),
            InlineKeyboardButton(f'Pelvis', callback_data="pelvisTarget"),
            InlineKeyboardButton(f'Left leg', callback_data="llegTarget")
        ],
        [
            InlineKeyboardButton(f'Right knee', callback_data="rkneeTarget"),
            InlineKeyboardButton(f'Left knee', callback_data="lkneeTarget")
        ],
        [
            InlineKeyboardButton(f'Right foot', callback_data="rfootTarget"),
            InlineKeyboardButton(f'Left foot', callback_data="lfootTarget")
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.callback_query.message.reply_text("Choose body part to attack:", reply_markup=reply_markup)

def combat_upper_direction(update: Update, context: CallbackContext) -> None:
    keyboard =[
        [InlineKeyboardButton(f'Head', callback_data="headTarget")],
        [
            InlineKeyboardButton(f'Right shoulder', callback_data="rshoulderTarget"),
            InlineKeyboardButton(f'Torso', callback_data="torsoTarget"),
            InlineKeyboardButton(f'Left shoulder', callback_data="lshoulderTarget")
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.callback_query.message.reply_text("Choose body part to attack:", reply_markup=reply_markup)
def skill_action(update: Update, context: CallbackContext) -> None:
    global enemy_grabbed
    global enemy_isGrabbed
    rightHand = user_equipment['rwield']
    leftHand = user_equipment['lwield']
    
    if enemy_grabbed['lhandGrabbed'] == 1 or enemy_grabbed['rhandGrabbed'] == 1 or enemy_grabbed['llegGrabbed'] == 1 or enemy_grabbed['rlegGrabbed'] == 1:
        enemy_isGrabbed = True
    else:
        enemy_isGrabbed = False

    keyboard = [
        [
            InlineKeyboardButton("Push", callback_data="pushDirection")
        ],
        [
            InlineKeyboardButton(f"Change position    (Current: {user_position})", callback_data="userSide")
        ]
    ]
    if (leftHand == None or rightHand == None) and (user_health['rhandHealth'] > 2 or user_health['lhandHealth'] > 2) and enemy_isGrabbed == False:
        keyboard.append([InlineKeyboardButton("Grab", callback_data="grabBodypart")])
    elif enemy_isGrabbed == True:
        keyboard.append([InlineKeyboardButton("Release", callback_data="releaseBodypart")])
    keyboard.append([InlineKeyboardButton("Back", callback_data="enterCombat")])

    print(enemy_isGrabbed)

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.callback_query.message.reply_text("I cannot injure by this... but I can win some time...", reply_markup=reply_markup)

def push_direction(update: Update, context: CallbackContext) -> None:
    keyboard = [
        [
            InlineKeyboardButton("Forward", callback_data="forwardPush")
        ],
        [
            InlineKeyboardButton("Left", callback_data="leftPush"),
            InlineKeyboardButton("Right", callback_data="rightPush")
        ],
        [
            InlineKeyboardButton("Backward", callback_data="backPush")
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.callback_query.message.reply_text("In which direction?", reply_markup=reply_markup)

def grab_action(update: Update, context: CallbackContext) -> None:
    keyboard = []

    if enemy_health['rhandHealth'] > 0 or enemy_health['relbowHealth'] > 0 or enemy_health['rshoulderHealth'] > 0:
        keyboard.append([InlineKeyboardButton("Right hand", callback_data="grabRightHand")])
    if enemy_health['lhandHealth'] > 0 or enemy_health['lelbowHealth'] > 0 or enemy_health['lshoulderHealth'] > 0:
        keyboard.append([InlineKeyboardButton("Left hand", callback_data="grabLeftHand")])
    if enemy_health['rlegHealth'] > 0 or enemy_health['rkneeHealth'] > 0 or enemy_health['rfootHealth'] > 0:
        keyboard.append([InlineKeyboardButton("Right leg", callback_data="grabRightLeg")])
    if enemy_health['llegHealth'] > 0 or enemy_health['lkneeHealth'] > 0 or enemy_health['lfootHealth'] > 0:
        keyboard.append([InlineKeyboardButton("Left leg", callback_data="grabLeftLeg")])

    keyboard.append([InlineKeyboardButton("Back", callback_data="enterCombat")])

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.callback_query.message.reply_text("Which body part of opponent?", reply_markup=reply_markup)
#ENEMY
def enemy_action(update: Update, context: CallbackContext) -> None:
    chat = update.effective_chat
    global dead_counter
    global turn_token
    global user_target
    global enemy_target
    global user_health
    global enemy_health
    global user_tdamage
    global enemy_tdamage

    if turn_token == 0:
        turn_token = 1
    else:
        turn_token = 0

    if  (enemy_health['headHealth'] <=5 or enemy_health['neckHealth'] <= 5) and (dead_counter == 0 or dead_counter == 1):
        context.bot.send_message(chat_id=chat.id,text = "It's furious!")
        enemy_attack = bite
        enemy_tdamage = enemy_attack['damage']
        enemy_target = 'neck'
        dead_counter += 1
    elif enemy_grabbed['rhandGrabbed'] == 1 or enemy_grabbed['lhandGrabbed'] == 1:
        context.bot.send_message(chat_id=chat.id,text = "Wait... IS IT SMILING!?")
        enemy_attack = bite
        enemy_tdamage = enemy_attack['damage']
        enemy_target = random.choice(enemy_crittarget_list)
    elif enemy_health['rshoulderHealth'] <= 5 or enemy_health['lshoulderHealth'] <= 5 or enemy_health['relbowHealth'] <= 5 or enemy_health['lelbowHealth'] <= 5 or enemy_health['rhandHealth'] <= 5 or enemy_health['lhandHealth'] <= 5:
        context.bot.send_message(chat_id=chat.id,text = "It's angry...")
        enemy_attack = kick
        enemy_tdamage = enemy_attack['damage']
        enemy_target = random.choice(enemy_target_list)
    else:
        context.bot.send_message(chat_id=chat.id,text = "It's calm... even bloodlustly playful...")
        enemy_attack = random.choice(easy_attack_list)
        enemy_tdamage = enemy_attack['damage']
        enemy_target = random.choice(enemy_target_list)    

    context.bot.send_message(chat_id=chat.id,text = """(Enemy's turn)
    
    It has made it's choice...""")
    
    if turn_token == 1:
        turn_summary(update, context)
    elif turn_token == 0:
        action(update, context)
    
#ACTION SUMMARY
def turn_summary(update: Update, context: CallbackContext) -> None:
    chat = update.effective_chat
    userLose = 0
    userDeath = 0
    userVictory = 0
    Tie = 0
    global current_location
    global user_position
    global turn_token
    global user_target
    global enemy_target
    global user_health
    global enemy_health
    global user_tdamage
    global enemy_tdamage
    global enemy_Balance
    global enemy_maxBalance
    global enemy_grabbed

    #print(user_tdamage)
    #print(user_target)
    #print(enemy_tdamage)
    #print(enemy_target)

    print(enemy_Balance)
    print(enemy_maxBalance)

    if enemy_health['rkneeHealth'] <= 2:
        enemy_maxBalance -= 1 
    if enemy_health['lkneeHealth'] <= 2:
        enemy_maxBalance -= 1

    if enemy_Balance == enemy_maxBalance:
        enemy_tdamage = 0
        enemy_Balance = 0
        enemy_grabbed['rhandGrabbed'] = 0
        enemy_grabbed['lhandGrabbed'] = 0
        enemy_grabbed['rlegGrabbed'] = 0
        enemy_grabbed['llegGrabbed'] = 0
        context.bot.send_message(chat_id=chat.id,text = 'With rumble your opponent fell down! You have time to strike!')
    elif enemy_Balance == enemy_maxBalance - 1:
        context.bot.send_message(chat_id=chat.id,text = 'Your opponent stagger... one more push...')
    elif enemy_Balance == enemy_maxBalance - 2:
        context.bot.send_message(chat_id=chat.id,text = "It's legs tremble!")

    if user_target == enemy_target:
        context.bot.send_message(chat_id=chat.id,text = 'Attack was parried!')
    else:
        if user_target == 'head':
            enemy_health['headHealth'] -= user_tdamage
            context.bot.send_message(chat_id=chat.id,text = f'Hit in head!    [{enemy_health["headHealth"]}]')
        elif user_target == 'neck':
            enemy_health['neckHealth'] -= user_tdamage
            context.bot.send_message(chat_id=chat.id,text = f'Hit in neck!    [{enemy_health["neckHealth"]}]')
        elif user_target == 'torso':
            enemy_health['torsoHealth'] -= user_tdamage
            context.bot.send_message(chat_id=chat.id,text = f'Hit in torso!    [{enemy_health["torsoHealth"]}]')
        elif user_target == 'pelvis':
            enemy_health['pelvisHealth'] -= user_tdamage
            context.bot.send_message(chat_id=chat.id,text = f'Hit in pelvis!    [{enemy_health["pelvisHealth"]}]')
        elif user_target == 'rshoulder':
            enemy_health['rshoulderHealth'] -= user_tdamage
            context.bot.send_message(chat_id=chat.id,text = f'Hit in right shoulder!    [{enemy_health["rshoulderHealth"]}]')
        elif user_target == 'lshoulder':
            enemy_health['lshoulderHealth'] -= user_tdamage
            context.bot.send_message(chat_id=chat.id,text = f'Hit in left shoulder!    [{enemy_health["lshoulderHealth"]}]')
        elif user_target == 'relbow':
            enemy_health['relbowHealth'] -= user_tdamage
            context.bot.send_message(chat_id=chat.id,text = f'Hit in right elbow!    [{enemy_health["relbowHealth"]}]')
        elif user_target == 'lelbow':
            enemy_health['lelbowHealth'] -= user_tdamage
            context.bot.send_message(chat_id=chat.id,text = f'Hit in left elbow!    [{enemy_health["lelbowHealth"]}]')
        elif user_target == 'rhand':
            enemy_health['rhandHealth'] -= user_tdamage
            context.bot.send_message(chat_id=chat.id,text = f'Hit in right hand!    [{enemy_health["rhandHealth"]}]')
        elif user_target == 'lhand':
            enemy_health['lhandHealth'] -= user_tdamage
            context.bot.send_message(chat_id=chat.id,text = f'Hit in left hand!    [{enemy_health["lhandHealth"]}]')
        elif user_target == 'rleg':
            enemy_health['rlegHealth'] -= user_tdamage
            context.bot.send_message(chat_id=chat.id,text = f'Hit in right leg!    [{enemy_health["rlegHealth"]}]')
        elif user_target == 'lleg':
            enemy_health['llegHealth'] -= user_tdamage
            context.bot.send_message(chat_id=chat.id,text = f'Hit in left leg!    [{enemy_health["llegHealth"]}]')
        elif user_target == 'rknee':
            enemy_health['rkneeHealth'] -= user_tdamage
            enemy_Balance += 1
            context.bot.send_message(chat_id=chat.id,text = f'Hit in right knee!    [{enemy_health["rkneeHealth"]}]')
        elif user_target == 'lknee':
            enemy_health['lkneeHealth'] -= user_tdamage
            enemy_Balance += 1
            context.bot.send_message(chat_id=chat.id,text = f'Hit in left knee!    [{enemy_health["lkneeHealth"]}]')
        elif user_target == 'rfoot':
            enemy_health['rfootHealth'] -= user_tdamage
            context.bot.send_message(chat_id=chat.id,text = f'Hit in right foot!    [{enemy_health["rfootHealth"]}]')
        elif user_target == 'lfoot':
            enemy_health['lfootHealth'] -= user_tdamage
            context.bot.send_message(chat_id=chat.id,text = f'Hit in left foot!    [{enemy_health["lfootHealth"]}]')

        if enemy_tdamage == 3:
            if enemy_target == 'head':
                user_health['headHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"It bite {user_data['name']} in head!    [{user_health['headHealth']}]")
            elif enemy_target == 'neck':
                user_health['neckHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"It bite {user_data['name']} in neck!    [{user_health['neckHealth']}]")
            elif enemy_target == 'torso':
                user_health['torsoHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"It bite {user_data['name']} in torso!    [{user_health['torsoHealth']}]")
            elif enemy_target == 'pelvis':
                user_health['pelvisHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"It bite {user_data['name']} in pelvis!    [{user_health['pelvisHealth']}]")
            elif enemy_target == 'rshoulder':
                user_health['rshoulderHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"It bite {user_data['name']} in right shoulder!    [{user_health['rshoulderHealth']}]")
            elif enemy_target == 'lshoulder':
                user_health['lshoulderHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"It bite {user_data['name']} in left shoulder!    [{user_health['lshoulderHealth']}]")
            elif enemy_target == 'relbow':
                user_health['relbowHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"It bite {user_data['name']} in right elbow!    [{user_health['relbowHealth']}]")
            elif enemy_target == 'lelbow':
                user_health['lelbowHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"It bite {user_data['name']} in left elbow!    [{user_health['lelbowHealth']}]")
            elif enemy_target == 'rhand':
                user_health['rhandHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"It bite {user_data['name']} in right hand!    [{user_health['rhandHealth']}]")
            elif enemy_target == 'lhand':
                user_health['lhandHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"It bite {user_data['name']} in left hand!    [{user_health['lhandHealth']}]")
            elif enemy_target == 'rleg':
                user_health['rlegHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"It bite {user_data['name']} in right leg!    [{user_health['rlegHealth']}]")
            elif enemy_target == 'lleg':
                user_health['llegHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"It bite {user_data['name']} in left leg!    [{user_health['llegHealth']}]")
            elif enemy_target == 'rknee':
                user_health['rkneeHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"It bite {user_data['name']} in right knee!    [{user_health['rkneeHealth']}]")
            elif enemy_target == 'lknee':
                user_health['lkneeHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"It bite {user_data['name']} in left knee!    [{user_health['lkneeHealth']}]")
            elif enemy_target == 'rfoot':
                user_health['rfootHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"It bite {user_data['name']} in right foot!    [{user_health['rfootHealth']}]")
            elif enemy_target == 'lfoot':
                user_health['lfootHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"It bite {user_data['name']} in left foot!    [{user_health['lfootHealth']}]")
        elif enemy_tdamage == 2:
            if enemy_target == 'head':
                user_health['headHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"Kick in {user_data['name']}'s head!    [{user_health['headHealth']}]")
            elif enemy_target == 'neck':
                user_health['neckHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"Kick in {user_data['name']}'s neck!    [{user_health['neckHealth']}]")
            elif enemy_target == 'torso':
                user_health['torsoHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"Kick in {user_data['name']}'s torso!    [{user_health['torsoHealth']}]")
            elif enemy_target == 'pelvis':
                user_health['pelvisHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"Kick in {user_data['name']}'s pelvis!    [{user_health['pelvisHealth']}]")
            elif enemy_target == 'rshoulder':
                user_health['rshoulderHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"Kick in {user_data['name']}'s right shoulder!    [{user_health['rshoulderHealth']}]")
            elif enemy_target == 'lshoulder':
                user_health['lshoulderHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"Kick in {user_data['name']}'s left shoulder!    [{user_health['lshoulderHealth']}]")
            elif enemy_target == 'relbow':
                user_health['relbowHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"Kick in {user_data['name']}'s right elbow!    [{user_health['relbowHealth']}]")
            elif enemy_target == 'lelbow':
                user_health['lelbowHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"Kick in {user_data['name']}'s left elbow!    [{user_health['lelbowHealth']}]")
            elif enemy_target == 'rhand':
                user_health['rhandHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"Kick in {user_data['name']}'s right hand!    [{user_health['rhandHealth']}]")
            elif enemy_target == 'lhand':
                user_health['lhandHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"Kick in {user_data['name']}'s left hand!    [{user_health['lhandHealth']}]")
            elif enemy_target == 'rleg':
                user_health['rlegHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"Kick in {user_data['name']}'s right leg!    [{user_health['rlegHealth']}]")
            elif enemy_target == 'lleg':
                user_health['llegHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"Kick in {user_data['name']}'s left leg!    [{user_health['llegHealth']}]")
            elif enemy_target == 'rknee':
                user_health['rkneeHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"Kick in {user_data['name']}'s right knee!    [{user_health['rkneeHealth']}]")
            elif enemy_target == 'lknee':
                user_health['lkneeHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"Kick in {user_data['name']}'s left knee!    [{user_health['lkneeHealth']}]")
            elif enemy_target == 'rfoot':
                user_health['rfootHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"Kick in {user_data['name']}'s right foot!    [{user_health['rfootHealth']}]")
            elif enemy_target == 'lfoot':
                user_health['lfootHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"Kick in {user_data['name']}'s left foot!    [{user_health['lfootHealth']}]")
        elif enemy_tdamage == 1:
            if enemy_target == 'head':
                user_health['headHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"Hit in {user_data['name']}'s head!    [{user_health['headHealth']}]")
            elif enemy_target == 'neck':
                user_health['neckHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"Hit in {user_data['name']}'s neck!    [{user_health['neckHealth']}]")
            elif enemy_target == 'torso':
                user_health['torsoHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"Hit in {user_data['name']}'s torso!    [{user_health['torsoHealth']}]")
            elif enemy_target == 'pelvis':
                user_health['pelvisHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"Hit in {user_data['name']}'s pelvis!    [{user_health['pelvisHealth']}]")
            elif enemy_target == 'rshoulder':
                user_health['rshoulderHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"Hit in {user_data['name']}'s right shoulder!    [{user_health['rshoulderHealth']}]")
            elif enemy_target == 'lshoulder':
                user_health['lshoulderHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"Hit in {user_data['name']}'s left shoulder!    [{user_health['lshoulderHealth']}]")
            elif enemy_target == 'relbow':
                user_health['relbowHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"Hit in {user_data['name']}'s right elbow!    [{user_health['relbowHealth']}]")
            elif enemy_target == 'lelbow':
                user_health['lelbowHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"Hit in {user_data['name']}'s left elbow!    [{user_health['lelbowHealth']}]")
            elif enemy_target == 'rhand':
                user_health['rhandHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"Hit in {user_data['name']}'s right hand!    [{user_health['rhandHealth']}]")
            elif enemy_target == 'lhand':
                user_health['lhandHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"Hit in {user_data['name']}'s left hand!    [{user_health['lhandHealth']}]")
            elif enemy_target == 'rleg':
                user_health['rlegHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"Hit in {user_data['name']}'s right leg!    [{user_health['rlegHealth']}]")
            elif enemy_target == 'lleg':
                user_health['llegHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"Hit in {user_data['name']}'s left leg!    [{user_health['llegHealth']}]")
            elif enemy_target == 'rknee':
                user_health['rkneeHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"Hit in {user_data['name']}'s right knee!    [{user_health['rkneeHealth']}]")
            elif enemy_target == 'lknee':
                user_health['lkneeHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"Hit in {user_data['name']}'s left knee!    [{user_health['lkneeHealth']}]")
            elif enemy_target == 'rfoot':
                user_health['rfootHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"Hit in {user_data['name']}'s right foot!    [{user_health['rfootHealth']}]")
            elif enemy_target == 'lfoot':
                user_health['lfootHealth'] -= enemy_tdamage
                context.bot.send_message(chat_id=chat.id,text = f"Hit in {user_data['name']}'s left foot!    [{user_health['lfootHealth']}]")
        elif enemy_tdamage == 0:
            context.bot.send_message(chat_id=chat.id,text = f"You've succesfuly interupted its attack!")


    if (user_health['headHealth'] <=2 and user_health['headHealth'] > 0) or (user_health['neckHealth'] <= 1 and user_health['neckHealth'] > 0) or (user_health['torsoHealth'] <= 3 and user_health['torsoHealth'] > 0) or (user_health['pelvisHealth'] <= 3 and user_health['pelvisHealth'] > 0):
        print("You lose")
        userLose += 1
    elif ((enemy_health['headHealth'] <=0) or (enemy_health['neckHealth'] <= 0) or (enemy_health['torsoHealth'] <= 0) or (enemy_health['pelvisHealth'] <= 0)) and ((user_health['headHealth'] <= 0) or (user_health['neckHealth'] <= 0) or (user_health['torsoHealth'] <= 0) or (user_health['pelvisHealth'] <= 0)):
        print("You both are dead")
        Tie += 1
    elif ((user_health['headHealth'] <= 0) or (user_health['neckHealth'] <= 0) or (user_health['torsoHealth'] <= 0) or (user_health['pelvisHealth'] <= 0)) and ((enemy_health['headHealth'] > 0) or (enemy_health['neckHealth'] > 0) or (enemy_health['torsoHealth'] > 0) or (enemy_health['pelvisHealth'] > 0)):
        print("You died")
        userDeath += 1
    elif ((enemy_health['headHealth'] <=0) or (enemy_health['neckHealth'] <= 0) or (enemy_health['torsoHealth'] <= 0) or (enemy_health['pelvisHealth'] <= 0)) and ((user_health['headHealth'] > 0) or (user_health['neckHealth'] > 0) or (user_health['torsoHealth'] > 0) or (user_health['pelvisHealth'] > 0)):
        print("You won")
        userVictory += 1
    
        

    #print(enemy_health['headHealth'])

    #print(turn_token)
    #print(userLose)
    #print(userDeath)
    #print(userVictory)

    user_target = ''
    enemy_target = ''

    if userVictory == 1:
        user_victory(update, context)
    elif userLose == 1 and userVictory != 1:
        user_defeat(update, context)
    elif userDeath == 1 and userVictory != 1:
        user_death(update, context)
    elif Tie == 1:
        tie(update,context)
    elif turn_token == 0:
        action(update, context)
    elif turn_token == 1:
        enemy_action(update, context)


#buttons reader
def button(update: Update, context: CallbackContext) -> None:
    global current_location
    global user_position
    global turn_token
    global user_target
    global enemy_target
    global user_health
    global enemy_health
    global user_tdamage
    global enemy_tdamage
    global enemy_Balance
    global enemy_maxBalance

    query = update.callback_query
    query.answer()

    if query.data == "startGame":
        query.edit_message_text(text=f"The Bridge behind fog... You wake up in the void...")
        start_game(update, context)
    elif query.data == "options":
        query.edit_message_text(text=f"Here are no options... by now...")
        back_main_menu(update, context)
    elif query.data == "credits":
        query.edit_message_text(text=f"Game made by PupetOne")
        back_main_menu(update, context)
    elif query.data == "backToMainMenu":
        query.edit_message_text(text=f"Gears tick back in time...")
        amenu(update, context)
    elif query.data == "newGame":
        query.edit_message_text(text=f"Memories flow in river... The golem in front of you takes huge ceramic tablet and scribes on it...")
        user_name(update, context)
    elif query.data == "loadGame":
        query.edit_message_text(text=f"WIP... please wait...")
        back_main_menu(update, context)    
    elif query.data == "male":
        query.edit_message_text(text=f"Alright, nice to meet you, sir...")
        user_data['sex'] = 'Male' 
        user_race(update, context)
    elif query.data == "female":
        query.edit_message_text(text=f"I'm glad to know you, lady...")   
        user_data['sex'] = 'Female'
        user_race(update, context)
    elif query.data == "humanRace":
        query.edit_message_text(text=f'Interesting... Could you describe us meaning of word "Humanity"?')  
        user_data['race'] = 'Human'
        starter_pack(update, context)
    elif query.data == "knightSet":
        query.edit_message_text(text=f"So, {user_data['name']}, you had enough money for preparations, how I see...")
        user_fin(update, context)
    elif query.data == "freelancerSet":
        query.edit_message_text(text=f"{user_data['name']}, free as birds, yes?")
        user_fin(update, context)
    elif query.data == "adventurerSet":
        query.edit_message_text(text=f"{user_data['name']}, it's never late to begin.")
        user_data['class'] = 'Adventurer'
        addItemToInventory()
        user_fin(update, context)
    elif query.data == "finishReg":
        query.edit_message_text(text=f"""Good... Remember, your name - {user_data['name']}... I'll be waiting with this tablet - proof of your existence in this world... Repeat it... Say, who you are...
        
        (Use '/myname' and input your character's name)""")
        save_data(update, context)
    elif query.data == "voidLoc":
        query.edit_message_text(text=f"Dark and bright. Cold and warm. Here is silent, but noises in your head get louder...")
        current_location = void
        #print(current_location)
        location_gen(update, context)
    elif query.data == "forestLoc":
        query.edit_message_text(text=f"Green leaves and sunlight breaking through the thick treetops... But you are not alone...")
        current_location = random.choice(list(forest_location_list))
        #print(current_location)
        location_gen(update, context)
    elif query.data == "dungeonLoc":
        query.edit_message_text(text=f"Long cold old dark tunnels of abandoned crypt... Maybe, that was not the best idea to come here...")
        current_location = random.choice(list(dungeon_location_list))
        #print(current_location)
        location_gen(update, context)
    elif query.data == "enterCombat":
        if turn_token == 0:
            query.edit_message_text(text=f"FIGHT AND SURVIVE!")
            action(update, context)
        else:
            query.edit_message_text(text=f"FIGHT AND SURVIVE!")
            enemy_action(update, context)
    elif query.data == "skillAction":
        query.edit_message_text(text=f"I need to trick it somehow...")
        skill_action(update, context)
    elif query.data == "pushDirection":
        query.edit_message_text(text=f"I need to trick it somehow...")
        push_direction(update, context)
    elif query.data == "forwardPush":
        if current_location['fwall'] == 1:    
            query.edit_message_text(text=f"Hit! It's legs now tremble!")
            enemy_Balance += 1
            if turn_token == 0:
                enemy_action(update, context)
            else:
                turn_summary(update, context)
        else:
            query.edit_message_text(text=f"I missed!")
            if turn_token == 0:
                enemy_action(update, context)
            else:
                turn_summary(update, context)
    elif query.data == "leftPush":
        if current_location['lwall'] == 1:    
            query.edit_message_text(text=f"Hit! It's legs now tremble!")
            enemy_Balance += 1
            if turn_token == 0:
                enemy_action(update, context)
            else:
                turn_summary(update, context)
        else:
            query.edit_message_text(text=f"I missed!")
            if turn_token == 0:
                enemy_action(update, context)
            else:
                turn_summary(update, context)
    elif query.data == "rightPush":
        if current_location['rwall'] == 1:    
            query.edit_message_text(text=f"Hit! It's legs now tremble!")
            enemy_Balance += 1
            if turn_token == 0:
                enemy_action(update, context)
            else:
                turn_summary(update, context)
        else:
            query.edit_message_text(text=f"I missed!")
            if turn_token == 0:
                enemy_action(update, context)
            else:
                turn_summary(update, context)
    elif query.data == "backPush":
        if current_location['bwall'] == 1:    
            query.edit_message_text(text=f"Hit! It's legs now tremble!")
            enemy_Balance += 1
            if turn_token == 0:
                enemy_action(update, context)
            else:
                turn_summary(update, context)
        else:
            query.edit_message_text(text=f"I missed!")
            if turn_token == 0:
                enemy_action(update, context)
            else:
                turn_summary(update, context)
    elif query.data == "grabBodypart":
        query.edit_message_text(text= "Grab...")
        grab_action(update, context)
    elif query.data == "grabRightHand":
        query.edit_message_text(text="Gotcha!")
        enemy_grabbed['rhandGrabbed'] = 1
        if turn_token == 0:
            enemy_action(update, context)
        else:
            turn_summary(update, context)
    elif query.data == "grabLeftHand":
        query.edit_message_text(text="Gotcha!")
        enemy_grabbed['lhandGrabbed'] = 1
        if turn_token == 0:
            enemy_action(update, context)
        else:
            turn_summary(update, context)
    elif query.data == "grabRightLeg":
        query.edit_message_text(text="Gotcha!")
        enemy_grabbed['rlegGrabbed'] = 1
        enemy_maxBalance = 1
        print(f'Max enemy balance: {enemy_maxBalance}')
        if turn_token == 0:
            enemy_action(update, context)
        else:
            turn_summary(update, context)
    elif query.data == "grabLeftLeg":
        query.edit_message_text(text="Gotcha!")
        enemy_grabbed['llegGrabbed'] = 1
        enemy_maxBalance = 1
        print(f'Max enemy balance: {enemy_maxBalance}')
        if turn_token == 0:
            enemy_action(update, context)
        else:
            turn_summary(update, context)
    elif query.data == "releaseBodypart":
        enemy_grabbed['rhandGrabbed'] = 0
        enemy_grabbed['lhandGrabbed'] = 0
        enemy_grabbed['rlegGrabbed'] = 0
        enemy_grabbed['llegGrabbed'] = 0
        enemy_maxBalance = 3
        print(f'Max enemy balance: {enemy_maxBalance}')
        if turn_token == 0:
            enemy_action(update, context)
        else:
            turn_summary(update, context)
    elif query.data == "userSide" and user_position == 'Right side':
        user_position = 'Left side'    
        query.edit_message_text(text=f"You swapped position. {user_position} will take hit on itself.")
        action(update, context)
    elif query.data == "userSide" and user_position == 'Left side':
        user_position = 'Right side'    
        query.edit_message_text(text=f"You swapped position. {user_position} will take hit on itself.")
        action(update, context)
    elif query.data == "combatAction":
        query.edit_message_text(text=f"I'll end it...")
        combat_hand(update, context)
    elif query.data == "rightHand":
        query.edit_message_text(text=f"How will I hit?")
        combat_rightHit(update, context)
    elif query.data == "leftHand":
        query.edit_message_text(text=f"How will I hit?")
        combat_leftHit(update, context)
    elif query.data == "rightDirectHit":
        query.edit_message_text(text=f"I'll cut it...")
        user_tdamage = user_equipment['rwield']['damage']
        #print(user_tdamage)
        combat_direction(update, context)
    elif query.data == "rightBluntHit":
        query.edit_message_text(text=f"I'll smack it...")
        user_tdamage = user_equipment['rwield']['damage'] - 2
        enemy_Balance += 1
        #print(user_tdamage)
        combat_direction(update, context)
    elif query.data == "rightUpperHit":
        query.edit_message_text(text=f"I'll crash it...")
        user_tdamage = user_equipment['rwield']['damage'] + 2
        #print(user_tdamage)
        combat_upper_direction(update, context)
    elif query.data == "leftDirectHit":
        query.edit_message_text(text=f"I'll cut it...")
        user_tdamage = user_equipment['lwield']['damage']
        #print(user_tdamage)
        combat_direction(update, context)
    elif query.data == "leftBluntHit":
        query.edit_message_text(text=f"I'll smack it...")
        user_tdamage = user_equipment['lwield']['damage'] - 2
        enemy_Balance += 1
        #print(user_tdamage)
        combat_upper_direction(update, context)
    elif query.data == "leftUpperHit":
        query.edit_message_text(text=f"I'll crash it...")
        user_tdamage = user_equipment['lwield']['damage'] + 2
        #print(user_tdamage)
        combat_upper_direction(update, context)
    elif query.data == "punch":
        query.edit_message_text(text=f"Get off!")
        user_tdamage = 1
        #print(user_tdamage)
        combat_direction(update, context)
    elif query.data == "headTarget":
        query.edit_message_text(text=f"Keep your eyes!")
        user_target = 'head'
        #print(user_target)
        if turn_token == 0:
            enemy_action(update, context)
        else:
            turn_summary(update, context)
    elif query.data == "neckTarget":
        query.edit_message_text(text=f"That'll be fast.")
        user_target = 'neck'
        #print(user_target)
        if turn_token == 0:
            enemy_action(update, context)
        else:
            turn_summary(update, context)
    elif query.data == "torsoTarget":
        query.edit_message_text(text=f"I'll not miss!")
        user_target = 'torso'
        #print(user_target)
        if turn_token == 0:
            enemy_action(update, context)
        else:
            turn_summary(update, context)
    elif query.data == "pelvisTarget":
        query.edit_message_text(text=f"I'm sorry...")
        user_target = 'pelvis'
        #print(user_target)
        if turn_token == 0:
            enemy_action(update, context)
        else:
            turn_summary(update, context)
    elif query.data == "rshoulderTarget":
        query.edit_message_text(text=f"Don't move!")
        user_target = 'rsoulder'
        #print(user_target)
        if turn_token == 0:
            enemy_action(update, context)
        else:
            turn_summary(update, context)
    elif query.data == "lshoulderTarget":
        query.edit_message_text(text=f"Don't move!")
        user_target = 'rshoulder'
        #print(user_target)
        if turn_token == 0:
            enemy_action(update, context)
        else:
            turn_summary(update, context)
    elif query.data == "relbowTarget":
        query.edit_message_text(text=f"It'll crack.")
        user_target = 'relbow'
        #print(user_target)
        if turn_token == 0:
            enemy_action(update, context)
        else:
            turn_summary(update, context)
    elif query.data == "lelbowTarget":
        query.edit_message_text(text=f"It'll crack")
        user_target = 'lelbow'
        #print(user_target)
        if turn_token == 0:
            enemy_action(update, context)
        else:
            turn_summary(update, context)
    elif query.data == "rhandTarget":
        query.edit_message_text(text=f"Drop your weapon!")
        user_target = 'rhand'
        #print(user_target)
        if turn_token == 0:
            enemy_action(update, context)
        else:
            turn_summary(update, context)
    elif query.data == "lhandTarget":
        query.edit_message_text(text=f"Drop your weapon!")
        user_target = 'lhand'
        #print(user_target)
        if turn_token == 0:
            enemy_action(update, context)
        else:
            turn_summary(update, context)
    elif query.data == "rlegTarget":
        query.edit_message_text(text=f"Fall!")
        user_target = 'rleg'
        #print(user_target)
        if turn_token == 0:
            enemy_action(update, context)
        else:
            turn_summary(update, context)
    elif query.data == "llegTarget":
        query.edit_message_text(text=f"Fall!")
        user_target = 'lleg'
        #print(user_target)
        if turn_token == 0:
            enemy_action(update, context)
        else:
            turn_summary(update, context)
    elif query.data == "rkneeTarget":
        query.edit_message_text(text=f"Tremble!")
        user_target = 'rknee'
        #print(user_target)
        if turn_token == 0:
            enemy_action(update, context)
        else:
            turn_summary(update, context)
    elif query.data == "lkneeTarget":
        query.edit_message_text(text=f"Tremble!")
        user_target = 'lknee'
        #print(user_target)
        if turn_token == 0:
            enemy_action(update, context)
        else:
            turn_summary(update, context)
    elif query.data == "rfootTarget":
        query.edit_message_text(text=f"You'll not run away!")
        user_target = 'rfoot'
        #print(user_target)
        if turn_token == 0:
            enemy_action(update, context)
        else:
            turn_summary(update, context)
    elif query.data == "lfootTarget":
        query.edit_message_text(text=f"You'll not run away!")
        user_target = 'lfoot'
        #print(user_target)
        if turn_token == 0:
            enemy_action(update, context)
        else:
            turn_summary(update, context)
    elif query.data == 'restartLoc':
        query.edit_message_text(text=f"Fight to death, Champion! Or win, hero...")
        restart_location(update, context)
    #elif query.data == "":
    #    query.edit_message_text(text=f"")

def user_defeat(update: Update, context: CallbackContext) -> None:
    chat = update.effective_chat
    context.bot.send_message(chat_id = chat.id, text = f"You passed out in the middle of a fight... You are still alive and enlightened...")
    keyboard = [
        [
            InlineKeyboardButton("Yes",callback_data='restartLoc')
        ],
        [
            InlineKeyboardButton("No", callback_data='backToMainMenu')
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.callback_query.message.reply_text("Retry?", reply_markup=reply_markup)

def user_death(update: Update, context: CallbackContext) -> None:
    chat = update.effective_chat
    context.bot.send_message(chat_id = chat.id, text = f"You died... Blood is warm?... You cannot die, no more... You were dead, but now you are enlightened...")
    keyboard = [
        [
            InlineKeyboardButton("Yes",callback_data='restartLoc')
        ],
        [
            InlineKeyboardButton("No", callback_data='backToMainMenu')
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.callback_query.message.reply_text("Retry?", reply_markup=reply_markup)

def user_victory(update: Update, context: CallbackContext) -> None:
    chat = update.effective_chat
    context.bot.send_message(chat_id = chat.id, text = f"Before it's fell down moveless, you saw a cleverness in it's fire eyes. Voices in your head sounded from its corpse: ''{user_data['name']}, thank you... I'm really sorry for this fight...''")
    keyboard = [
        [
            InlineKeyboardButton("Yes",callback_data='restartLoc')
        ],
        [
            InlineKeyboardButton("No", callback_data='backToMainMenu')
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.callback_query.message.reply_text("Retry?", reply_markup=reply_markup)

def tie(update: Update, context: CallbackContext) -> None:
    chat = update.effective_chat
    context.bot.send_message(chat_id = chat.id, text = f"You are both lying on the ground... And suddenly you've feel that this puppet is similar to you... No, you're wrong...")
    keyboard = [
        [
            InlineKeyboardButton("Yes",callback_data='restartLoc')
        ],
        [
            InlineKeyboardButton("No", callback_data='backToMainMenu')
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.callback_query.message.reply_text("Retry?", reply_markup=reply_markup)

#def echo(update: Update, context: CallbackContext):
#    context.bot.send_message(chat_id=update.effective_chat.id, text=update.message.text)

#Credits
def back_main_menu(update: Update, context: CallbackContext) -> None:
    keyboard = [
        [
            InlineKeyboardButton("Back", callback_data="backToMainMenu")
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.callback_query.message.reply_text("Back to main menu?", reply_markup=reply_markup)

#Credits
def back_main_amenu(update: Update, context: CallbackContext) -> None:
    keyboard = [
        [
            InlineKeyboardButton("Back", callback_data="backToMainMenu")
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.message.reply_text("Back to main menu?", reply_markup=reply_markup)


def stop(update: Update, context: CallbackContext):
    context.bot.send_message(chat_id=update.effective_chat.id, text="Shutting down...")
    updater.stop()

def start_callback(context: telegram.ext.CallbackContext):
    context.bot.send_message(chat_id=CHAT_ID, text="""Golem was awaken...
    
    *Type '/start' to begin*""")

def player_reply(update, context):
    user_says = ''.join(context.args)
    update.message.reply_text(f"You said: {user_says}")

job_minute = job_queue.run_once(start_callback, 1)

#Start bot
start_handler = CommandHandler('start', start)
dispatcher.add_handler(start_handler)

#Main menu
menu_handler = CommandHandler("menu", menu)
dispatcher.add_handler(menu_handler)

amenu_handler = CommandHandler("amenu", amenu)
dispatcher.add_handler(amenu_handler)

start_game_handler = CommandHandler("start_game", start_game)
dispatcher.add_handler(start_game_handler)

#User sex
user_sex_handler = CommandHandler("user_sex", user_sex)
dispatcher.add_handler(user_sex_handler)
#User race
user_race_handler = CommandHandler("user_race", user_race)
dispatcher.add_handler(user_race_handler)
#User starter pack
user_spack_handler = CommandHandler("user_spack", starter_pack)
dispatcher.add_handler(user_spack_handler)
#User registration finish
userReg_fin_handler = CommandHandler("userReg_fin", user_fin)
dispatcher.add_handler(userReg_fin_handler)

back_main_menu_handler = CommandHandler("menu", back_main_menu)
dispatcher.add_handler(back_main_menu_handler)
#Buttons
dispatcher.add_handler(CallbackQueryHandler(button))


location_handler = CommandHandler("locgen", adventure)
dispatcher.add_handler(location_handler)
#echo
#echo_handler = MessageHandler(Filters.text & (~Filters.command), echo)
#dispatcher.add_handler(echo_handler)

#Turn off bot
stop_handler = CommandHandler('stop', stop)
dispatcher.add_handler(stop_handler)

#User says something
dispatcher.add_handler(CommandHandler("say", player_reply))

dispatcher.add_handler(CommandHandler("myname", adventure))
#Save data
def save_data(update: Update, context: CallbackContext):
    
    with open(f"E:\git\GolemBot\{user_data['name']}.csv", "w", encoding='UTF8', newline='') as f:

        fieldnames = ['name', 'sex', 'race', 'class']

        writer = csv.writer(f)
        dwriter = csv.DictWriter(f, fieldnames=fieldnames)
        #
        dwriter.writerow(user_data)
        writer.writerow(user_inventory)

updater.start_polling(timeout=10)
updater.idle()


{'fwall': 0, 'rwall': 1, 'lwall': 0, 'bwall': 0}
{'fwall': 0, 'rwall': 0, 'lwall': 1, 'bwall': 1}


2023-01-10 12:37:16,953 - apscheduler.scheduler - INFO - Adding job tentatively -- it will be properly scheduled when the scheduler starts
2023-01-10 12:37:16,956 - apscheduler.scheduler - INFO - Added job "start_callback" to job store "default"
2023-01-10 12:37:16,957 - apscheduler.scheduler - INFO - Scheduler started


1686756740:AAGwTrm-o91o17n3-DwkTUxJ2Mnrvwy1kCE
720434978
Opened database successfully


2023-01-10 12:37:17,884 - apscheduler.executors.default - INFO - Running job "start_callback (trigger: date[2023-01-10 10:37:17 UTC], next run at: 2023-01-10 10:37:17 UTC)" (scheduled at 2023-01-10 10:37:17.869710+00:00)
2023-01-10 12:37:17,886 - apscheduler.scheduler - INFO - Removed job 41659798a272447386afd667f189957a
2023-01-10 12:37:18,639 - apscheduler.executors.default - INFO - Job "start_callback (trigger: date[2023-01-10 10:37:17 UTC], next run at: 2023-01-10 10:37:17 UTC)" executed successfully


In [1]:
import sqlite3

conn = sqlite3.connect('test.db')
print("Opened database successfully")

conn.execute('''CREATE TABLE USERS
         (ID INT PRIMARY KEY NOT NULL,
         NAME           TEXT    NOT NULL,
         SEX            TEXT     NOT NULL,
         RACE        TEXT       NOT NULL,
         CLASS         TEXT     NOT NULL);''')
print("Table created successfully")

conn.close()

Opened database successfully
Table created successfully


In [3]:
import sqlite3

conn = sqlite3.connect('test.db')

print("Opened database successfully")

#conn.execute("INSERT INTO USERS (NAME,SEX,RACE,CLASS) \
#      VALUES ('Paul', 'Male', 'Human', 'Adventurer' )");

#conn.execute("INSERT INTO USERS (NAME,SEX,RACE,CLASS) \
#      VALUES ('Allen', 'Male', 'Human', 'Adventurer' )");

#conn.execute("INSERT INTO USERS (NAME,SEX,RACE,CLASS) \
#      VALUES ('Rose', 'Female', 'Human', 'Adventurer')");

#conn.execute("INSERT INTO USERS (NAME,SEX,RACE,CLASS) \
#      VALUES ('Mark', 'Male', 'Human', 'Adventurer')");

conn.execute("UPDATE USERS set CLASS = 'Adventurer' where ID = ")
conn.commit()
print("Total number of rows updated :", conn.total_changes)

conn.execute("DELETE from USERS where ID = 1;")
conn.commit()
print("Total number of rows deleted :", conn.total_changes)

conn.commit()
print("Records created successfully")
conn.close()

Opened database successfully


OperationalError: incomplete input

In [2]:
words = ['potato','kiwi','apple','print']
trigger = True
wordList = []
answerList = []

for i in words:
    if len(i) == 5 and trigger == True:
        trigger = False
        for j in i:
            wordList.append(j)

questionWord = (f'{wordList[0]}{wordList[1]}{wordList[2]}{wordList[3]}{wordList[4]}')

def game():
    answer = input(str('Try to guess 5char word:'))
    if len(answer) == 5 and (answer, str):
        for i in answer:
            answerList.append(i)
        check(answerList,wordList)
    else: 
        print('Use 5char word!')
        game()

def check(alist,qlist):
    for i in range(len(wordList)):
        if alist[i] == qlist[i]:
            print('!')
        elif alist[i] in qlist:
            print('?')
        else:
            print('_')
    if questionWord == f'{answerList[0]}{answerList[1]}{answerList[2]}{answerList[3]}{answerList[4]}':
        print(f'YOU WIN')
    else:
        answerList.clear()
        print('Try again')
        game()
game()

!
!
!
_
!
Try again
!
!
?
!
?
Try again


KeyboardInterrupt: Interrupted by user